Computational use of the `controls` module implementing the example given in [1].

[1] Doyle, John. "Guaranteed Margins for LQG Regulators," IEEE Transactions on Automatic Control, 1978

In [1]:
import control
import numpy
import math

In [2]:
A = np.array([ [1,1], [0,1]])
Bu = np.array([ [0,1] ]).T
Bw = np.array([ [1,1] ]).T
C = np.array([[1,0]])

In [3]:
q =1
sigma = 1.0
Q = q*np.array([[1, 1]]).T@np.array([[1,1]])
# This controller Q feels arbitray to me, but may have been choosen
# to mimic the 
# Q for observer design
R = 1

In [4]:
# state feedback gain
g,_,_ = control.lqr(A,Bu,Q,R)
g=g.T
# equivalent to 
# X,_,_=control.care(A,Bu,Q,R)
# g = X@Bu@np.linalg.inv(R)

# observer gain
k,_,_ = control.lqr(A.T,C.T,Bw@Bw.T*sigma,R)
k=k.T

In [5]:
# closed form gains from [1]
#f = 2+math.sqrt(4+q)
#d = 2+math.sqrt(4+sigma)
#
#g = f*np.array([1,1])[:,np.newaxis]
#k = d*np.array([1,1])[:,np.newaxis]

Setup system as two interconnected systems

In [6]:
plant = control.ss( A, Bu, C, 0 )

Observer
$$
\begin{align}
u &= -G\hat{x}\\
\dot{\hat{x}} &= A \hat{x} + Bu + -K(\hat{y}-y)\\
&= (A-BG-KC)\hat{x} + Ky
\end{align}
$$

In [7]:
obs = control.ss( A-Bu@g.T-k@C, k, -g.T, np.zeros((1,1)))

In [8]:
system = control.feedback(plant,obs,sign=1)

In [9]:
system.A

matrix([[ 1.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        , -4.23606798, -4.23606798],
        [ 4.23606798,  0.        , -3.23606798,  1.        ],
        [ 4.23606798,  0.        , -8.47213595, -3.23606798]])

Closed loop system is stable.

In [10]:
np.linalg.eigvals(system.A)

array([-1.61803423, -1.61803375, -0.61803421, -0.61803377])

In [11]:
#perturbed system
m = 1.1
# m*plant is a gain of m on the input of the system plant
perturbed_system = control.feedback(m*plant, obs, sign=1)

Closed loop system is no longer stable.

In [12]:
np.linalg.eigvals(perturbed_system.A)

array([-3.0756458 +0.j        , -0.74371711+1.51229927j,
       -0.74371711-1.51229927j,  0.09094406+0.j        ])